In [3]:
import numpy as np
import torch

In [4]:
# Conigs
HIGH_BIT = 64
LOW_BIT = 32
BITS = 32
PRECISION = 13
LOW_TYPE = np.uint32
HIGH_TYPE = np.uint64

MOD = 2 ** HIGH_BIT
P = 37

In [9]:
class RSS:
    def __init__(self, st, rd, mod):
        self.first = st
        self.second = rd
        self.mod = mod

def float2fxp(fl_x, f):
    int_x = fl_x * (2 ** f)
    ring_x = int_x % MOD

def RSS_share(x, mod):
    x1 = np.random.randint(MOD, dtype='uint64')
    x2 = np.random.randint(MOD, dtype='uint64')
    x3 = (x - x1 - x2)
    return RSS(x1, x2, mod), RSS(x2, x3, mod), RSS(x3, x1, mod)

def RSS_reveal(x_s, mod):   
    rs1, rs2, rs3 = x_s
    return rs1.first + rs2.first + rs3.first

def reduction(x_s):
    rs1, rs2, rs3 = x_s
    rs1_low = RSS(rs1.first.astype(LOW_TYPE), rs1.second.astype(LOW_TYPE), 2**LOW_BIT)
    rs2_low = RSS(rs2.first.astype(LOW_TYPE), rs2.second.astype(LOW_TYPE), 2**LOW_BIT)
    rs3_low = RSS(rs3.first.astype(LOW_TYPE), rs3.second.astype(LOW_TYPE), 2**LOW_BIT)
    return rs1_low, rs2_low, rs3_low

def extension(x_s):
    rs1, rs2, rs3 = x_s
    assert rs1.first.dtype == LOW_TYPE
    rs1_low = RSS(rs1.first.astype(HIGH_TYPE), rs1.second.astype(HIGH_TYPE), 2**HIGH_BIT)
    rs2_low = RSS(rs2.first.astype(HIGH_TYPE), rs2.second.astype(HIGH_TYPE), 2**HIGH_BIT)
    rs3_low = RSS(rs3.first.astype(HIGH_TYPE), rs3.second.astype(HIGH_TYPE), 2**HIGH_BIT)
    return rs1_low, rs2_low, rs3_low
    
def truncate(x_s):
    rs1, rs2, rs3 = x_s
    rs1_low = RSS(rs1.first.astype(np.uint32), rs1.second.astype(np.uint32), 2**LOW_BIT)
    rs2_low = RSS(rs2.first.astype(np.uint32), rs2.second.astype(np.uint32), 2**LOW_BIT)
    rs3_low = RSS(rs3.first.astype(np.uint32), rs3.second.astype(np.uint32), 2**LOW_BIT)
    return rs1_low, rs2_low, rs3_low


################   HELPER FUNCTIONS    ################
def print_rs(x_rs):
    print(f'first: {x_rs.first}, second: {x_rs.second}')

def print_xs(x_s):
    rs1, rs2, rs3 = x_s
    print_rs(rs1)
    print_rs(rs2)
    print_rs(rs3)



In [11]:
x = np.uint64(10)
x_s = RSS_share(x, mod=MOD)
print_xs(x_s)
print(RSS_reveal(x_s, MOD))

x_s_red = reduction(x_s)
print_xs(x_s_red)
print(RSS_reveal(x_s_red, mod=2**LOW_BIT))

x_s_red_ext = extension(x_s_red)
print_xs(x_s_red_ext)
print(RSS_reveal(x_s_red_ext, mod=2**HIGH_BIT))
print(RSS_reveal(x_s_red_ext, mod=2**HIGH_BIT) % (2 ** LOW_BIT))

first: 12289829493103614557, second: 9723484994372770474
first: 9723484994372770474, second: 14880173659942718211
first: 14880173659942718211, second: 12289829493103614557
10
first: 1183799901, second: 3429767850
first: 3429767850, second: 3976366851
first: 3976366851, second: 1183799901
10
first: 1183799901, second: 3429767850
first: 3429767850, second: 3976366851
first: 3976366851, second: 1183799901
8589934602
4294967306.0


<ipython-input-9-c46425a9f2fa>:14: RuntimeWarning: overflow encountered in ulong_scalars
  x3 = (x - x1 - x2)
<ipython-input-9-c46425a9f2fa>:19: RuntimeWarning: overflow encountered in ulong_scalars
  return rs1.first + rs2.first + rs3.first
<ipython-input-9-c46425a9f2fa>:19: RuntimeWarning: overflow encountered in uint_scalars
  return rs1.first + rs2.first + rs3.first
